In [ ]:
# Plots of received data in 1M PHY: Phase and Phase difference 

import math
import cmath
import json
import numpy as np
import matplotlib.pyplot as plt

### Methods
def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

data = {
    "IQ":[[2,-12],[11,2],[-4,10],
        [-11,-5],[5,-10],[9,6],
        [-8,9],[-9,-7],[7,8],
        [-8,-10],[5,10],[-4,-12],
        [2,11],[-2,-12],[-2,11],
        [3,-11],[-5,10],[5,-10],
        [-7,9],[8,-9],[-9,7],
        [10,-6],[-11,4],[11,-4],
        [-12,1],[11,1],[-12,-2],
        [10,3],[-11,-5],[9,6],
        [-10,-7],[7,8],[-6,-10],
        [5,9],[-4,-11],[1,11],
        [-1,-12],[-1,11],[3,-12],
        [-5,10],[5,-11],[-7,9],
        [7,-9],[-9,6],[9,-6],
        [-11,3],[10,-3],[-12,0],
        [11,1],[-12,-2],[10,3],
        [-11,-6],[9,5],[-9,-8],
        [7,8],[-7,-10],[4,10],
        [-5,-11],[1,11],[0,-12],
        [-2,11],[2,-12],[-4,11],
        [6,-10],[-8,8],[8,-9],
        [-9,7],[9,-6],[-11,4],
        [11,-3],[-12,1],[11,-1],
        [-12,-3],[10,2],[-10,-6],
        [10,5],[-8,-8],[7,9],
        [-6,-11],[5,10],[-3,-12],
        [1,11]]}

    Phases = []
    PhasesDiff = []
    for i in range(len(data['IQ'])-1):
        iq_next = complex(data['IQ'][i + 1][0], data['IQ'][i + 1][1])
        iq_current = complex(data['IQ'][i][0], data['IQ'][i][1])
        phase_next = np.rad2deg(np.arctan2(iq_next.imag, iq_next.real))
        phase_current = np.rad2deg(np.arctan2(iq_current.imag, iq_current.real))
        Phases.append(phase_current)
        PhasesDiff.append((to_plus_minus_pi(phase_next - phase_current)))

    print(Phases)
    plt.figure(figsize=(16, 10), dpi=100)
    plt.plot(Phases)
    plt.show()

    print(PhasesDiff)
    plt.figure(figsize=(16, 10), dpi=100)
    plt.plot(PhasesDiff)
    plt.show()

In [ ]:
# Path test
import pathlib

pathlib.Path("logs").mkdir(exist_ok=True)
print(pathlib.Path().absolute())


In [ ]:
# Plots of received data in 1M PHY: Phase and Phase difference (SIMPLE HORIZONTAL/VERTICAL PATTERN)

import math
import cmath
import json
import numpy as np
import matplotlib.pyplot as plt

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

data = []

# Load data with only valid IQ samples from JSON log  

# with open('../logs/outdoor/2022.10.23/log_2022.10.23_13.49.34.json') as f: # 0 deg TRASH
with open('../logs/outdoor/2022.10.23/log_2022.10.23_13.52.22.json') as f: # 30 deg OK
# with open('../logs/outdoor/2022.10.23/log_2022.10.23_13.54.57.json') as f: # 45 deg
# with open('../logs/outdoor/2022.10.23/log_2022.10.23_13.57.36.json') as f: # 60 deg
# with open('../logs/outdoor/2022.10.23/log_2022.10.23_13.59.39.json') as f: # 90 deg
# with open('../logs/outdoor/2022.10.23/log_2022.10.23_14.01.51.json') as f: # 120 deg
# with open('../logs/outdoor/2022.10.23/log_2022.10.23_14.09.12.json') as f: # 135 deg
# with open('../logs/outdoor/2022.10.23/log_2022.10.23_14.26.24.json') as f: # 150 deg
# with open('../logs/indoor/2022.11.04/log_2022.11.04_22.24.54.json') as f:  #90 deg indoor
# with open('../logs/indoor/2022.11.04/log_2022.11.04_22.39.17.json') as f:  #110 deg indoor
# with open('../logs/indoor/2022.11.04/log_2022.11.04_22.43.31.json') as f:  #140 deg indoor
    log = json.load(f)
    record_data = {
        "Header": log["Header"],
        "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
    }
    data.append(record_data)

results = []

# Iterate through each connection event 
for n in range(len(data[0]["Records"])-1):
    channel = data[0]["Records"][n]["Channel"]
    
    IQ = []
    result = []

    # Iterate through each IQ sample to normalize it 
    for elem in data[0]["Records"][n]["IQ"]:
        I = elem[0]/128 if elem[0] < 0 else elem[0]/127
        Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
        IQ.append(complex(I,Q))

    refPeriodDiff = []
    sampPeriodDiff = []
    refMeanDiff = 0

    # Iterate through each sample (82) in one connection event 
    for i in range(len(IQ)-1):
        # Phase calculation from IQ samples
        phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
        phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

        if (i < 7):
            refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
        elif (i == 7):
            refMeanDiff = calc_mean(refPeriodDiff)
        else:
            sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

    # Calculate azimuth angles
    inverted = False
    for i in range(len(sampPeriodDiff)-1):
        # Skip difference between last and first measurement in upper and lower section
        if ((i + 1) % 4 == 0):
            # Invert in lower section due to opposite scanning direction
            inverted = not inverted
        else:
            calc_angle(sampPeriodDiff[i], inverted, result, channel_2_freq(channel))

    results.append(calc_mean(result))

    print(f"# {n} Channel: {channel} Freq: {channel_2_freq(channel)}")
    print(f"(11-12) Diff: {sampPeriodDiff[0]} Angle: {result[0]}")
    print(f"(12-01) Diff: {sampPeriodDiff[1]} Angle: {result[1]}")
    print(f"(01-02) Diff: {sampPeriodDiff[2]} Angle: {result[2]}")

    print(f"(05-06) Diff: {sampPeriodDiff[3]} Angle: {result[3]}")
    print(f"(06-07) Diff: {sampPeriodDiff[4]} Angle: {result[4]}")
    print(f"(07-08) Diff: {sampPeriodDiff[5]} Angle: {result[5]}")

    print(f"(11-12) Diff: {sampPeriodDiff[6]} Angle: {result[6]}")
    print(f"(12-01) Diff: {sampPeriodDiff[7]} Angle: {result[7]}")
    print(f"(01-02) Diff: {sampPeriodDiff[8]} Angle: {result[8]}")

    print(f"(05-06) Diff: {sampPeriodDiff[9]} Angle: {result[9]}")
    print(f"(06-07) Diff: {sampPeriodDiff[10]} Angle: {result[10]}")
    print(f"(07-08) Diff: {sampPeriodDiff[11]} Angle: {result[11]}")

# print(results)
# print(calc_mean(results))

# Plot phases
# plt.figure(figsize=(16, 10), dpi=100)
# plt.plot(Phases)
# plt.show()

# Plot reference difference of phases
# plt.figure(figsize=(16, 10), dpi=100)
# plt.plot(refPeriodDiff)
# plt.show()

# Plot sampling difference of phases
# plt.figure(figsize=(16, 10), dpi=100)
# plt.plot(sampPeriodDiff)
# plt.show()

# TODO It looks like eliminating 6th sample from each measurement would improve accuracy as difference between 8 and 7th antenna is invalid


In [ ]:
# Plots of received data in 1M and 2M PHY: Phase and Phase difference (COMPLEX PATTERN)
import math
import cmath
import json
import numpy as np
import matplotlib.pyplot as plt

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

data = []

# Load data with only valid IQ samples from JSON log  
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.48.07.json') as f:  # 60deg max height
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.33.48.json') as f:  # 60deg mid height
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.07.55.json') as f:  # 60deg min height
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.50.06.json') as f:  # 90deg max height
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.32.02.json') as f:  # 90deg mid height
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.09.56.json') as f:  # 90deg min height
with open('../logs/outdoor/2022.10.31/log_2022.10.31_16.51.51.json') as f:  # 45deg min height 2M PHY
        
    log = json.load(f)
    record_data = {
        "Header": log["Header"],
        "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
    }
    data.append(record_data)

results_azimuth = []
results_elevation = []

# Iterate through each connection event 
for n in range(len(data[0]["Records"])-1):
    channel = data[0]["Records"][n]["Channel"]
    IQ = []

    # Iterate through each IQ sample to normalize it 
    for elem in data[0]["Records"][n]["IQ"]:
        I = elem[0]/128 if elem[0] < 0 else elem[0]/127
        Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
        IQ.append(complex(I,Q))

    refPeriodDiff = []
    sampPeriodDiff = []
    refMeanDiff = 0

    # Iterate through each sample (82) in one connection event 
    for i in range(len(IQ)-1):
        
        # Phase calculation from IQ samples
        phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
        phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

        if (i < 7):
            refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
        elif (i == 7):
            refMeanDiff = calc_mean(refPeriodDiff)
        else:
            sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

    azimuth = []
    elevation = []
    inverted = False
    horizontal = True

    # Calculate direction angles
    for i in range(len(sampPeriodDiff)-1):
        # Skip corner differences samples
        if ((i + 1) % 4 == 0):
            if(not horizontal): 
                # Change to inverted after 2 sections of non inverted data due to opposite scanning direction
                inverted = not inverted
            # Change orientation after each section 
            horizontal = not horizontal
        else:
            if (horizontal):
                calc_angle(sampPeriodDiff[i], inverted, azimuth, channel_2_freq(channel))
            else:
                calc_angle(sampPeriodDiff[i], inverted, elevation, channel_2_freq(channel))

    results_azimuth.append(calc_mean(azimuth))
    results_elevation.append(calc_mean(elevation))

print(f"Azimuth mean: {calc_mean(results_azimuth)}")
print(f"Elevation mean: {calc_mean(results_elevation)}")

In [ ]:
# Plots of received data in 1M and 2M PHY: Phase and Phase difference (COMPLEX PATTERN)

import math
import cmath
import json
from mpl_toolkits.axisartist.axislines import AxesZero
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

data = []

# Load data with only valid IQ samples from JSON log  
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_16.51.51.json') as f:  # 45deg min height 2M PHY
with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.05.16.json') as f:  # 45deg min height 1M PHY
# with open('../logs/outdoor/2022.10.31/log_2022.10.31_15.03.12.json') as f:  # 30deg min height 1M PHY
    
        
    log = json.load(f)
    record_data = {
        "Header": log["Header"],
        "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
    }
    data.append(record_data)

results_azimuth = []
results_elevation = []

# Iterate through each connection event 
for n in range(len(data[0]["Records"])-1):
    IQ = []
    channel = data[0]["Records"][n]["Channel"]

    # Iterate through each IQ sample to normalize it 
    for elem in data[0]["Records"][n]["IQ"]:
        I = elem[0]/128 if elem[0] < 0 else elem[0]/127
        Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
        IQ.append(complex(I,Q))

    refPeriodDiff = []
    sampPeriodDiff = []
    refMeanDiff = 0

    # Iterate through each sample (82) in one connection event 
    for i in range(len(IQ)-1):
        # Phase calculation from IQ samples
        phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
        phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

        if (i < 7):
            refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
        elif (i == 7):
            refMeanDiff = calc_mean(refPeriodDiff)
        else:
            # 2*refMeanDiff as a in ref period sample is taken twice as fast as in sampling period, 
            # so it needs to be doubled
            sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

    azimuth = []
    elevation = []
    inverted = False
    horizontal = True

    # Calculate direction angles
    for i in range(len(sampPeriodDiff)-1):
        # Skip corner differences samples
        if ((i + 1) % 4 == 0):
            if(not horizontal): 
                # Change to inverted after 2 sections of non inverted data due to opposite scanning direction
                inverted = not inverted
            # Change orientation after each section 
            horizontal = not horizontal
        else:
            if (horizontal):
                calc_angle(sampPeriodDiff[i], inverted, azimuth, channel_2_freq(channel))
            else:
                calc_angle(sampPeriodDiff[i], inverted, elevation, channel_2_freq(channel))

    results_azimuth.append(calc_mean(azimuth))
    results_elevation.append(calc_mean(elevation))

x = [30, 45, 60, 90, 135, 150]
y = [30, 45, 60, 90, 135, 150]

def animate(i):
    # out.set_xdata(results_azimuth[i]-90)
    # out.set_ydata(results_elevation[i]-90)
    out.set_xdata(x[i]-90)
    out.set_ydata(y[i]-90)
    return out,

fig = plt.figure(figsize=(16, 10), dpi=100)
ax = fig.add_subplot(axes_class=AxesZero)
out, = ax.plot(results_azimuth[i]-90, results_elevation[i]-90,'ro')

for direction in ["xzero", "yzero"]:
    # adds X and Y-axis from the origin
    ax.axis[direction].set_visible(True)

for direction in ["left", "right", "bottom", "top"]:
    # hides borders
    ax.axis[direction].set_visible(False)
    
ax.grid(True)
ax.set_title("Direction radar")
ax.set_xlim(-90,90)
ax.set_ylim(-90,90)
ticks = np.arange(-90,91,30)
ax.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax.yaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])

plt.tight_layout()
plt.show()


In [ ]:
# Animation

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# %matplotlib notebook

fig, ax = plt.subplots()

x = np.arange(0, 2*np.pi, 0.01)
line, = ax.plot(x, np.sin(x))


def animate(i):
    line.set_ydata(np.sin(x + i / 50))  # update the data.
    return line,


ani = animation.FuncAnimation(
    fig, animate, interval=20, blit=True, save_count=50)

# To save the animation, use e.g.
#
# ani.save("movie.mp4")
#
# or
#
# writer = animation.FFMpegWriter(
#     fps=15, metadata=dict(artist='Me'), bitrate=1800)
# ani.save("movie.mp4", writer=writer)

plt.show()

In [ ]:
# Outdoor accuracy 1M PHY

import math
import cmath
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

data = []

with open('../logs/outdoor/2022.11.15/log_2022.11.15_10.58.00.json') as f:
    log = json.load(f)
    record_data = {
        "Header": log["Header"],
        "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
    }
    data.append(record_data)

results_azimuth = []
results_elevation = []

# Iterate through each connection event 
for n in range(len(data[0]["Records"])-1):
    IQ = []
    channel = data[0]["Records"][n]["Channel"]

    # Iterate through each IQ sample to normalize it 
    for elem in data[0]["Records"][n]["IQ"]:
        I = elem[0]/128 if elem[0] < 0 else elem[0]/127
        Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
        IQ.append(complex(I,Q))

    refPeriodDiff = []
    sampPeriodDiff = []
    refMeanDiff = 0

    # Iterate through each sample (82) in one connection event 
    for i in range(len(IQ)-1):
        # Phase calculation from IQ samples
        phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
        phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

        if (i < 7):
            refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
        elif (i == 7):
            refMeanDiff = calc_mean(refPeriodDiff)
        else:
            # 2*refMeanDiff as a in ref period sample is taken twice as fast as in sampling period, 
            # so it needs to be doubled
            sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

    azimuth = []
    elevation = []
    inverted = False
    horizontal = True

    # Calculate direction angles
    for i in range(len(sampPeriodDiff)-1):
        # Skip corner differences samples
        if ((i + 1) % 4 == 0):
            if(not horizontal): 
                # Change to inverted after 2 sections of non inverted data due to opposite scanning direction
                inverted = not inverted
            # Change orientation after each section 
            horizontal = not horizontal
        else:
            if (horizontal):
                calc_angle(sampPeriodDiff[i], inverted, azimuth, channel_2_freq(channel))
            else:
                calc_angle(sampPeriodDiff[i], inverted, elevation, channel_2_freq(channel))

    results_azimuth.append(calc_mean(azimuth)-90)
    results_elevation.append(calc_mean(elevation)-90)

# print(f"Azimuth min: {min(results_azimuth)}, max: {max(results_azimuth)}")
# print(f"Elevation min: {min(results_elevation)}, max: {max(results_elevation)}")

# print(abs(math.tan(np.deg2rad(max(results_azimuth)))*4))
# print(abs(math.tan(np.deg2rad(min(results_azimuth)))*4))
# print(abs(math.tan(np.deg2rad(max(results_elevation)))*4))
# print(abs(math.tan(np.deg2rad(min(results_elevation)))*4))

# print(abs(math.tan(np.deg2rad(min(results_azimuth)))*4) + abs(math.tan(np.deg2rad(max(results_azimuth)))*4))
# print(abs(math.tan(np.deg2rad(min(results_elevation)))*4) + abs(math.tan(np.deg2rad(max(results_elevation)))*4))

# print(f"Azimuth min: {min(results_azimuth)}, max: {max(results_azimuth)}")
# print(f"Elevation min: {min(results_elevation)}, max: {max(results_elevation)}")

print(f"Samples count: {len(results_azimuth)}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
# fig.suptitle("Accuracy of outdoor measurement for transmitter located at (0,0)", fontsize='20')
ax1.grid(True)
ax1.set_xlim(-90,90)
ax1.set_ylim(-90,90)
ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)  
ax1.spines['left'].set_position('zero')
ax1.spines['bottom'].set_position('zero')
ax1.xaxis.set_label_coords(0, -0.08)
ax1.yaxis.set_label_coords(-0.08, 0)
ax1.set_ylabel('Elevation [deg]', fontsize='30')
ax1.set_xlabel('Azimuth [deg]', fontsize='30')
ticks = np.arange(-90,91,30)
ax1.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax1.yaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax1.tick_params(labelsize=30)
ax1.plot(results_azimuth, results_elevation, 'ro')

ax2.grid(True)
ax2.set_xlim(-4,4)
ax2.set_ylim(-4,4)
ax2.spines["right"].set_visible(False)
ax2.spines["top"].set_visible(False)  
ax2.spines['left'].set_position('zero')
ax2.spines['bottom'].set_position('zero')
ticks = np.arange(-4,5,2)
ax2.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax2.yaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax2.tick_params(labelsize=30)
ax2.plot(results_azimuth, results_elevation, 'ro', markersize=10)

ax1.add_patch(plt.Rectangle((-4, -4), 8, 8,
             edgecolor = 'k',
             facecolor = 'None',
             lw=2))
             
con1 = ConnectionPatch(xyA=(-4,4), xyB=(-4,4), coordsA="data", coordsB="data",
                      axesA=ax2, axesB=ax1, color="k")

con2 = ConnectionPatch(xyA=(-4,-4), xyB=(-4,-4), coordsA="data", coordsB="data",
                      axesA=ax2, axesB=ax1, color="k")

ax1.add_artist(con1)
ax1.add_artist(con2)

plt.subplots_adjust(left=0.07,
                    bottom=0.11,
                    right=0.95,
                    top=0.95,
                    wspace=0.15,
                    hspace=0.9)
# plt.show()  
plt.savefig("Accuracy_outdoor.pdf")

mad_azim = median(abs(results_azimuth-median(results_azimuth)))
mad_elev = median(abs(results_elevation-median(results_elevation)))

horizontal_accuracy = abs(math.tan(np.deg2rad(min(results_azimuth)))*4) + abs(math.tan(np.deg2rad(max(results_azimuth)))*4)
vertical_accuracy =  abs(math.tan(np.deg2rad(min(results_elevation)))*4) + abs(math.tan(np.deg2rad(max(results_elevation)))*4)

print(abs(math.tan(np.deg2rad(max(results_azimuth)))*4))
print(abs(math.tan(np.deg2rad(min(results_azimuth)))*4))
print(abs(math.tan(np.deg2rad(max(results_elevation)))*4))
print(abs(math.tan(np.deg2rad(min(results_elevation)))*4))
print(f"Horizontal accuracy: {horizontal_accuracy*100:.2f}")
print(f"Vertical accuracy: {vertical_accuracy*100:.2f}")
print(f"Max Azimuth: {max(results_azimuth):.2f}")
print(f"Min Azimuth: {min(results_azimuth):.2f}")
print(f"Median Azimuth: {median(results_azimuth):.2f}")
print(f"MAD Azimuth: {mad_azim:.2f}")
print(f"Max Elevation: {max(results_elevation):.2f}")
print(f"Min Elevation: {min(results_elevation):.2f}")
print(f"Median Elevation: {median(results_elevation):.2f}")
print(f"MAD Elevation: {mad_elev:.2f}")

In [ ]:
# Indoor accuracy 1M PHY

import math
import cmath
import json
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import numpy as np



###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

data = []

with open('../logs/outdoor/2022.11.23/log_2022.11.23_00.08.01.json') as f:
    log = json.load(f)
    record_data = {
        "Header": log["Header"],
        "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
    }
    data.append(record_data)

results_azimuth = []
results_elevation = []

# Iterate through each connection event 
for n in range(len(data[0]["Records"])-1):
    IQ = []
    channel = data[0]["Records"][n]["Channel"]

    # Iterate through each IQ sample to normalize it 
    for elem in data[0]["Records"][n]["IQ"]:
        I = elem[0]/128 if elem[0] < 0 else elem[0]/127
        Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
        IQ.append(complex(I,Q))

    refPeriodDiff = []
    sampPeriodDiff = []
    refMeanDiff = 0

    # Iterate through each sample (82) in one connection event 
    for i in range(len(IQ)-1):
        # Phase calculation from IQ samples
        phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
        phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

        if (i < 7):
            refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
        elif (i == 7):
            refMeanDiff = calc_mean(refPeriodDiff)
        else:
            # 2*refMeanDiff as a in ref period sample is taken twice as fast as in sampling period, 
            # so it needs to be doubled
            sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

    azimuth = []
    elevation = []
    inverted = False
    horizontal = True

    # Calculate direction angles
    for i in range(len(sampPeriodDiff)-1):
        # Skip corner differences samples
        if ((i + 1) % 4 == 0):
            if(not horizontal): 
                # Change to inverted after 2 sections of non inverted data due to opposite scanning direction
                inverted = not inverted
            # Change orientation after each section 
            horizontal = not horizontal
        else:
            if (horizontal):
                calc_angle(sampPeriodDiff[i], inverted, azimuth, channel_2_freq(channel))
            else:
                calc_angle(sampPeriodDiff[i], inverted, elevation, channel_2_freq(channel))

    results_azimuth.append(calc_mean(azimuth)-90)
    results_elevation.append(calc_mean(elevation)-90)

# print(abs(math.tan(np.deg2rad(max(results_azimuth)))*4))
# print(abs(math.tan(np.deg2rad(min(results_azimuth)))*4))
# print(abs(math.tan(np.deg2rad(max(results_elevation)))*4))
# print(abs(math.tan(np.deg2rad(min(results_elevation)))*4))

# print(f"Azimuth min: {min(results_azimuth)}, max: {max(results_azimuth)}")
# print(f"Elevation min: {min(results_elevation)}, max: {max(results_elevation)}")
print(f"Samples count: {len(results_azimuth)}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
# fig.suptitle("Accuracy of indoor measurement for transmitter located at (0,0)", fontsize='20')

ax1.grid(True)
ax1.set_xlim(-90,90)
ax1.set_ylim(-90,90)
ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)  
ax1.spines['left'].set_position('zero')
ax1.spines['bottom'].set_position('zero')
ax1.xaxis.set_label_coords(0, -0.08)
ax1.yaxis.set_label_coords(-0.07, 0)
ax1.set_ylabel('Elevation [deg]', fontsize='30')
ax1.set_xlabel('Azimuth [deg]', fontsize='30')
ticks = np.arange(-90,91,30)
ax1.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax1.yaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax1.tick_params(labelsize=30)
ax1.plot(results_azimuth, results_elevation, 'bo')

ax2.grid(True)
ax2.set_xlim(-4,4)
ax2.set_ylim(-4,4)
ax2.spines["right"].set_visible(False)
ax2.spines["top"].set_visible(False)  
ax2.spines['left'].set_position('zero')
ax2.spines['bottom'].set_position('zero')
ticks = np.arange(-4,5,2)
ax2.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax2.yaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax2.tick_params(labelsize=30)
ax2.plot(results_azimuth, results_elevation, 'bo', markersize=10)

ax1.add_patch(plt.Rectangle((-4, -4), 8, 8,
             edgecolor = 'k',
             facecolor = 'None',
             lw=2))
             
con1 = ConnectionPatch(xyA=(-4,4), xyB=(-4,4), coordsA="data", coordsB="data",
                      axesA=ax2, axesB=ax1, color="k")

con2 = ConnectionPatch(xyA=(-4,-4), xyB=(-4,-4), coordsA="data", coordsB="data",
                      axesA=ax2, axesB=ax1, color="k")
ax1.add_artist(con1)
ax1.add_artist(con2)

plt.subplots_adjust(left=0.07,
                    bottom=0.11,
                    right=0.95,
                    top=0.95,
                    wspace=0.15,
                    hspace=0.9)
plt.savefig("Accuracy_indoor.pdf")

mad_azim = median(abs(results_azimuth-median(results_azimuth)))
mad_elev = median(abs(results_elevation-median(results_elevation)))

horizontal_accuracy = abs(math.tan(np.deg2rad(min(results_azimuth)))*4) + abs(math.tan(np.deg2rad(max(results_azimuth)))*4)
vertical_accuracy =  abs(math.tan(np.deg2rad(min(results_elevation)))*4) + abs(math.tan(np.deg2rad(max(results_elevation)))*4)

print(abs(math.tan(np.deg2rad(max(results_azimuth)))*4))
print(abs(math.tan(np.deg2rad(min(results_azimuth)))*4))
print(abs(math.tan(np.deg2rad(max(results_elevation)))*4))
print(abs(math.tan(np.deg2rad(min(results_elevation)))*4))
print(f"Horizontal accuracy: {horizontal_accuracy*100:.2f}")
print(f"Vertical accuracy: {vertical_accuracy*100:.2f}")
print(f"Max Azimuth: {max(results_azimuth):.2f}")
print(f"Min Azimuth: {min(results_azimuth):.2f}")
print(f"Median Azimuth: {median(results_azimuth):.2f}")
print(f"MAD Azimuth: {mad_azim:.2f}")
print(f"Max Elevation: {max(results_elevation):.2f}")
print(f"Min Elevation: {min(results_elevation):.2f}")
print(f"Median Elevation: {median(results_elevation):.2f}")
print(f"MAD Elevation: {mad_elev:.2f}")

In [ ]:
# Plot 1M PHY: Phase difference

import math
import cmath
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

data = []

with open('../logs/outdoor/2022.11.21/log_2022.11.21_19.33.49.json') as f:
    log = json.load(f)
    record_data = {
        "Header": log["Header"],
        "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
    }
    data.append(record_data)

results_azimuth = []
results_elevation = []

IQ = []

# Iterate through each IQ sample to normalize it 
for elem in data[0]["Records"][10]["IQ"]:
    I = elem[0]/128 if elem[0] < 0 else elem[0]/127
    Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
    IQ.append(complex(I,Q))

Diff = []

# Iterate through each sample (82) in one connection event 
for i in range(len(IQ)-1):
    # Phase calculation from IQ samples
    phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
    phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))
    # Diff.append(to_plus_minus_pi(phaseNext - phaseCurrent))
    if(phaseNext - phaseCurrent < 0):
        Diff.append(phaseNext - phaseCurrent + 360)
    else:
        Diff.append(phaseNext - phaseCurrent)


fig = plt.figure(figsize=(20, 10), dpi=100)
ax = fig.add_subplot()

ax.grid(True)
ax.set_ylim(0,205)
ax.xaxis.set_label_coords(0, -0.08)
ax.yaxis.set_label_coords(-0.08, 0)
ax.set_ylabel('Phase difference [deg]', fontsize='30')
ax.set_xlabel('Sample [n]', fontsize='30')
ax.tick_params(labelsize=30)
# ax.set_title("Phase difference deviation from ideal values in 1M PHY (Single antenna)", fontsize='20')
plt.hlines(y=180, xmin=7, xmax=80, colors='green', linestyles='--', lw=5, label='Sampling period - 180 deg phase difference')
plt.hlines(y=90, xmin=0, xmax=6, colors='red', linestyles='--', lw=5, label='Reference period - 90 deg phase difference')
plt.legend(loc='lower right', fontsize=30)
ax.plot(Diff, 'bo', markersize=10)
# plt.tight_layout()
# plt.show()
plt.savefig("1M_PHY_Phase_Diff.pdf")


In [ ]:
# Plot 2M PHY: Phase difference

import math
import cmath
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

data = []

with open('../logs/outdoor/2022.11.21/log_2022.11.21_19.36.58.json') as f:
    log = json.load(f)
    record_data = {
        "Header": log["Header"],
        "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
    }
    data.append(record_data)

results_azimuth = []
results_elevation = []

IQ = []

# Iterate through each IQ sample to normalize it 
for elem in data[0]["Records"][10]["IQ"]:
    I = elem[0]/128 if elem[0] < 0 else elem[0]/127
    Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
    IQ.append(complex(I,Q))

Diff = []

# Iterate through each sample (82) in one connection event 
for i in range(len(IQ)-1):
    # Phase calculation from IQ samples
    phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
    phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))
    # Diff.append(to_plus_minus_pi(phaseNext - phaseCurrent))
    if(phaseNext - phaseCurrent < 20):
        Diff.append(phaseNext - phaseCurrent + 360)
    else:
        Diff.append(phaseNext - phaseCurrent)


fig = plt.figure(figsize=(20, 10), dpi=100)
ax = fig.add_subplot()

ax.grid(True)
ax.set_ylim(0,385)
ax.xaxis.set_label_coords(0, -0.08)
ax.yaxis.set_label_coords(-0.08, 0)
ax.set_ylabel('Phase difference [deg]', fontsize='30')
ax.set_xlabel('Sample [n]', fontsize='30')
ax.tick_params(labelsize=30)
# ax.set_title("Phase difference deviation from ideal values in 2M PHY (Single antenna)", fontsize='20')
plt.hlines(y=360, xmin=7, xmax=80, colors='green', linestyles='--', lw=5, label='Sampling period - 360 deg phase difference')
plt.hlines(y=180, xmin=0, xmax=6, colors='blue', linestyles='--', lw=5, label='Reference period - 180 deg phase difference')
# plt.legend(bbox_to_anchor=(0.27,0.2), loc="center left", borderaxespad=0, fontsize=18)
plt.legend(loc='lower right', fontsize=30)
ax.plot(Diff, 'ro', markersize=10)
plt.savefig("2M_PHY_Phase_Diff.pdf")


In [ ]:
# Plots of received data in 1M and 2M PHY: Phase and Phase difference (COMPLEX PATTERN)
from operator import attrgetter
import math
import cmath
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from statistics import *

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

max_val = ["log_2022.10.31_15.42.39.json", "log_2022.10.31_15.44.27.json", 
        "log_2022.10.31_15.46.15.json", "log_2022.10.31_15.48.07.json", 
        "log_2022.10.31_15.50.06.json", "log_2022.10.31_15.52.18.json", 
        "log_2022.10.31_15.54.14.json", "log_2022.10.31_15.57.26.json"]

mid_val = ["log_2022.10.31_15.40.15.json", "log_2022.10.31_15.37.15.json", 
        "log_2022.10.31_15.35.32.json", "log_2022.10.31_15.33.48.json", 
        "log_2022.10.31_15.32.02.json", "log_2022.10.31_15.29.53.json", 
        "log_2022.10.31_15.28.10.json", "log_2022.10.31_15.26.08.json"]

min_val = ["log_2022.10.31_15.00.48.json", "log_2022.10.31_15.03.12.json", 
        "log_2022.10.31_15.05.16.json", "log_2022.10.31_15.07.55.json", 
        "log_2022.10.31_15.09.56.json", "log_2022.10.31_15.12.23.json", 
        "log_2022.10.31_15.14.25.json", "log_2022.10.31_15.16.16.json"]

combined_azim = []
combined_elev = []

for log in min_val:

    data = []
    with open(f"../logs/outdoor/2022.10.31/{log}") as f:
        log = json.load(f)
        record_data = {
            "Header": log["Header"],
            "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
        }
        data.append(record_data)

    results_azimuth = []
    results_elevation = []

    # Iterate through each connection event 
    for n in range(len(data[0]["Records"])-1):
        IQ = []
        channel = data[0]["Records"][n]["Channel"]

        # Iterate through each IQ sample to normalize it 
        for elem in data[0]["Records"][n]["IQ"]:
            I = elem[0]/128 if elem[0] < 0 else elem[0]/127
            Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
            IQ.append(complex(I,Q))

        refPeriodDiff = []
        sampPeriodDiff = []
        refMeanDiff = 0

        # Iterate through each sample (82) in one connection event 
        for i in range(len(IQ)-1):
            # Phase calculation from IQ samples
            phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
            phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

            if (i < 7):
                refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
            elif (i == 7):
                refMeanDiff = calc_mean(refPeriodDiff)
            else:
                # 2*refMeanDiff as a in ref period sample is taken twice as fast as in sampling period, 
                # so it needs to be doubled
                sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

        azimuth = []
        elevation = []
        inverted = False
        horizontal = True

        # Calculate direction angles
        for i in range(len(sampPeriodDiff)-1):
            # Skip corner differences samples
            if ((i + 1) % 4 == 0):
                if(not horizontal): 
                    # Change to inverted after 2 sections of non inverted data due to opposite scanning direction
                    inverted = not inverted
                # Change orientation after each section 
                horizontal = not horizontal
            else:
                if (horizontal):
                    calc_angle(sampPeriodDiff[i], inverted, azimuth, channel_2_freq(channel))
                else:
                    calc_angle(sampPeriodDiff[i], inverted, elevation, channel_2_freq(channel))

        results_azimuth.append(calc_mean(azimuth)-90)
        results_elevation.append(calc_mean(elevation)-90)

    combined_azim.append(results_azimuth)
    combined_elev.append(results_elevation)

# Plot for azimuth range
fig = plt.figure(figsize=(12, 12), dpi=100)
ax = fig.add_subplot()
ax.grid(True)
ax.set_xlim(-90,91)
ax.set_ylim(-90,91)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.xaxis.set_label_coords(0, -0.05)
ax.yaxis.set_label_coords(-0.05, 0)
ax.set_ylabel('Elevation [deg]', fontsize='20')
ax.set_xlabel('Azimuth [deg]', fontsize='20')
ax.tick_params(labelsize=20)
ticks = np.arange(-90,92,30)
ax.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax.yaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
# ax.set_title("Antenna matrix azimuth range measurements", fontsize='20')
plot_ticks = [-90, -60, -45, -30, 0, 30, 45, 60]

for i in range(0, len(min_val)):
    plt.plot(combined_azim[i], combined_elev[i], 'o', label=f'TX located at {plot_ticks[i]} azimuth')
plt.legend(loc='best', fontsize='20')

for i in range(0, len(min_val)):
    mad_azim = median(abs(combined_azim[i]-median(combined_azim[i])))
    mad_elev = median(abs(combined_elev[i]-median(combined_elev[i])))
    sd_azim = stdev(combined_azim[i])
    sd_elev = stdev(combined_elev[i])
    print(f"{i} Max Azimuth: {max(combined_azim[i]):.2f}")
    print(f"{i} Min Azimuth: {min(combined_azim[i]):.2f}")
    print(f"{i} Median Azimuth: {median(combined_azim[i]):.2f}")
    print(f"{i} MAD Azimuth: {mad_azim:.2f}")

plt.subplots_adjust(left=0.07,
                    bottom=0.08,
                    right=0.95,
                    top=0.97,
                    wspace=0.15,
                    hspace=0.9)
# plt.tight_layout()
# plt.show()
plt.savefig("Azimuth_range.pdf")

In [204]:
# Plots of received data in 1M and 2M PHY: Phase and Phase difference (COMPLEX PATTERN)
from operator import attrgetter
import math
import cmath
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from statistics import *

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

max_val = ["log_2022.10.31_15.42.39.json", "log_2022.10.31_15.44.27.json", 
        "log_2022.10.31_15.46.15.json", "log_2022.10.31_15.48.07.json", 
        "log_2022.10.31_15.50.06.json", "log_2022.10.31_15.52.18.json", 
        "log_2022.10.31_15.54.14.json", "log_2022.10.31_15.57.26.json"]

mid_val = ["log_2022.10.31_15.40.15.json", "log_2022.10.31_15.37.15.json", 
        "log_2022.10.31_15.35.32.json", "log_2022.10.31_15.33.48.json", 
        "log_2022.10.31_15.32.02.json", "log_2022.10.31_15.29.53.json", 
        "log_2022.10.31_15.28.10.json", "log_2022.10.31_15.26.08.json"]

min_val = ["log_2022.10.31_15.00.48.json", "log_2022.10.31_15.03.12.json", 
        "log_2022.10.31_15.05.16.json", "log_2022.10.31_15.07.55.json", 
        "log_2022.10.31_15.09.56.json", "log_2022.10.31_15.12.23.json", 
        "log_2022.10.31_15.14.25.json", "log_2022.10.31_15.16.16.json"]

combined_azim = []
combined_elev = []

for log in min_val:

    data = []
    with open(f"../logs/outdoor/2022.10.31/{log}") as f:
        log = json.load(f)
        record_data = {
            "Header": log["Header"],
            "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
        }
        data.append(record_data)

    results_azimuth = []
    results_elevation = []

    # Iterate through each connection event 
    for n in range(len(data[0]["Records"])-1):
        IQ = []
        channel = data[0]["Records"][n]["Channel"]

        # Iterate through each IQ sample to normalize it 
        for elem in data[0]["Records"][n]["IQ"]:
            I = elem[0]/128 if elem[0] < 0 else elem[0]/127
            Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
            IQ.append(complex(I,Q))

        refPeriodDiff = []
        sampPeriodDiff = []
        refMeanDiff = 0

        # Iterate through each sample (82) in one connection event 
        for i in range(len(IQ)-1):
            # Phase calculation from IQ samples
            phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
            phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

            if (i < 7):
                refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
            elif (i == 7):
                refMeanDiff = calc_mean(refPeriodDiff)
            else:
                # 2*refMeanDiff as a in ref period sample is taken twice as fast as in sampling period, 
                # so it needs to be doubled
                sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

        azimuth = []
        elevation = []
        inverted = False
        horizontal = True

        # Calculate direction angles
        for i in range(len(sampPeriodDiff)-1):
            # Skip corner differences samples
            if ((i + 1) % 4 == 0):
                if(not horizontal): 
                    # Change to inverted after 2 sections of non inverted data due to opposite scanning direction
                    inverted = not inverted
                # Change orientation after each section 
                horizontal = not horizontal
            else:
                if (horizontal):
                    calc_angle(sampPeriodDiff[i], inverted, azimuth, channel_2_freq(channel))
                else:
                    calc_angle(sampPeriodDiff[i], inverted, elevation, channel_2_freq(channel))

        results_azimuth.append(calc_mean(azimuth)-90)
        results_elevation.append(calc_mean(elevation)-90)

    combined_azim.append(results_azimuth)
    combined_elev.append(results_elevation)

# Plot for azimuth range
fig = plt.figure(figsize=(8, 8), dpi=100)
ax = fig.add_subplot()
ax.grid(True)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
ax.spines['left'].set_position('zero')
ax.set_xlim(-95,95)
ax.xaxis.set_label_coords(0, -0.05)
ax.yaxis.set_label_coords(-0.08, 0)
ax.set_ylabel('Median Absolute Deviation [deg]', fontsize='18')
ax.set_xlabel('Azimuth [deg]', fontsize='18')
ax.tick_params(labelsize=18)
ticks = np.arange(-90,92,30)
ax.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
# ax.set_title("Antenna matrix azimuth range measurements", fontsize='20')
plot_ticks = [-90, -60, -45, -30, 0, 30, 45, 60]

for i in range(0, len(min_val)):
    mad_azim = median(abs(combined_azim[i]-median(combined_azim[i])))
    plt.plot(plot_ticks[i], mad_azim, 'ro', markersize=14)

# for i in range(0, len(min_val)):
#     mad_azim = median(abs(combined_azim[i]-median(combined_azim[i])))
#     mad_elev = median(abs(combined_elev[i]-median(combined_elev[i])))
#     sd_azim = stdev(combined_azim[i])
#     sd_elev = stdev(combined_elev[i])
#     print(f"{i} Max Azimuth: {max(combined_azim[i]):.2f}")
#     print(f"{i} Min Azimuth: {min(combined_azim[i]):.2f}")
#     print(f"{i} Median Azimuth: {median(combined_azim[i]):.2f}")
#     print(f"{i} MAD Azimuth: {mad_azim:.2f}")

plt.subplots_adjust(left=0.11,
                    bottom=0.08,
                    right=0.95,
                    top=0.97,
                    wspace=0.15,
                    hspace=0.9)
# plt.tight_layout()
# plt.show()
plt.savefig("MAD_range.pdf")

[-61.73351453784097, -69.16758833446482, -60.87276487354045, -74.95640439942638, -67.51777626112894, -63.5411310622724, -59.545045340508864, -64.99424330479374, -58.927542381934, -59.37259371104682, -65.26354284485399, -64.98135595274007, -57.82955203013636, -60.85607923836493, -70.34205608829052, -70.01450276576522, -63.72543197507528, -57.16294773106358, -59.52033482097328, -69.59944734665143, -64.26863206021653, -72.63952212817932, -63.33322311857482, -71.78467907221162, -69.81551321816977, -58.91374248571903, -62.38062483330934, -65.62126069672142, -66.51630185022752, -63.18322788616843, -63.73833052379614, -69.58665513108005, -49.311883360918316, -68.47511347262304, -65.24114854933494, -58.91178033001487, -63.56412100297572, -61.32901318336906, -63.77157801729214, -49.83407851387699, -61.68785592709831, -61.21718346458511, -58.48307067925995, -61.05567618907743, -72.63504270009227, -65.57058115841014, -66.96136289119688, -63.73329525710126, -68.58312156176585, -68.91542147597441, 

In [39]:
# Plots of received data in 1M and 2M PHY: Phase and Phase difference (COMPLEX PATTERN)
from operator import attrgetter
import math
import cmath
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from statistics import *

###################################################################
### Constants

vel_air = 299702547 # m/s
d = 0.05

###################################################################
### Methods

def to_plus_minus_pi(angle):
    while angle >= 180:
        angle -= 2 * 180
    while angle < -180:
        angle += 2 * 180
    return angle

def calc_mean(values):
    return sum(values)/len(values)

def calc_angle(deg_value, inverted, results, freq):
    wave_len = vel_air/freq
    output = 0
    try:
        result = math.acos((np.deg2rad(deg_value)*wave_len)/(2*math.pi*d))
    except ValueError:
        pass
    else:
        if inverted:
            output = abs(to_plus_minus_pi(np.rad2deg(result)+180))
        else:
            output = abs(to_plus_minus_pi(np.rad2deg(result)))
        results.append(output)

def channel_2_freq(channel):
    freq = 2450 #MHz
    if(channel <= 10):
        freq = 2404 + channel*2
    elif(channel < 37):
        freq = 2428 + (channel-11)*2
    elif(channel == 37):
        freq = 2402
    elif(channel == 38):
        freq = 2426
    else:
        freq = 2480
    return freq*1000000

###################################################################

max_val = ["log_2022.10.31_15.42.39.json", "log_2022.10.31_15.44.27.json", 
        "log_2022.10.31_15.46.15.json", "log_2022.10.31_15.48.07.json", 
        "log_2022.10.31_15.50.06.json", "log_2022.10.31_15.52.18.json", 
        "log_2022.10.31_15.54.14.json", "log_2022.10.31_15.57.26.json"]

mid_val = ["log_2022.10.31_15.40.15.json", "log_2022.10.31_15.37.15.json", 
        "log_2022.10.31_15.35.32.json", "log_2022.10.31_15.33.48.json", 
        "log_2022.10.31_15.32.02.json", "log_2022.10.31_15.29.53.json", 
        "log_2022.10.31_15.28.10.json", "log_2022.10.31_15.26.08.json"]

min_val = ["log_2022.10.31_15.00.48.json", "log_2022.10.31_15.03.12.json", 
        "log_2022.10.31_15.05.16.json", "log_2022.10.31_15.07.55.json", 
        "log_2022.10.31_15.09.56.json", "log_2022.10.31_15.12.23.json", 
        "log_2022.10.31_15.14.25.json", "log_2022.10.31_15.16.16.json"]

combined_azim = []
combined_elev = []

for log in min_val:

    data = []
    with open(f"../logs/outdoor/2022.10.31/{log}") as f:
        log = json.load(f)
        record_data = {
            "Header": log["Header"],
            "Records": [elem for elem in log["Records"] if len(elem["IQ"]) > 80]
        }
        data.append(record_data)

    results_azimuth = []
    results_elevation = []

    # Iterate through each connection event 
    for n in range(len(data[0]["Records"])-1):
        IQ = []
        channel = data[0]["Records"][n]["Channel"]

        # Iterate through each IQ sample to normalize it 
        for elem in data[0]["Records"][n]["IQ"]:
            I = elem[0]/128 if elem[0] < 0 else elem[0]/127
            Q = elem[1]/128 if elem[1] < 0 else elem[1]/127
            IQ.append(complex(I,Q))

        refPeriodDiff = []
        sampPeriodDiff = []
        refMeanDiff = 0

        # Iterate through each sample (82) in one connection event 
        for i in range(len(IQ)-1):
            # Phase calculation from IQ samples
            phaseCurrent = np.rad2deg(np.arctan2(IQ[i].imag, IQ[i].real))
            phaseNext = np.rad2deg(np.arctan2(IQ[i + 1].imag, IQ[i + 1].real))

            if (i < 7):
                refPeriodDiff.append(abs(to_plus_minus_pi(phaseNext - phaseCurrent)))
            elif (i == 7):
                refMeanDiff = calc_mean(refPeriodDiff)
            else:
                # 2*refMeanDiff as a in ref period sample is taken twice as fast as in sampling period, 
                # so it needs to be doubled
                sampPeriodDiff.append(to_plus_minus_pi((phaseNext - phaseCurrent) - 2*refMeanDiff))

        azimuth = []
        elevation = []
        inverted = False
        horizontal = True

        # Calculate direction angles
        for i in range(len(sampPeriodDiff)-1):
            # Skip corner differences samples
            if ((i + 1) % 4 == 0):
                if(not horizontal): 
                    # Change to inverted after 2 sections of non inverted data due to opposite scanning direction
                    inverted = not inverted
                # Change orientation after each section 
                horizontal = not horizontal
            else:
                if (horizontal):
                    calc_angle(sampPeriodDiff[i], inverted, azimuth, channel_2_freq(channel))
                else:
                    calc_angle(sampPeriodDiff[i], inverted, elevation, channel_2_freq(channel))

        results_azimuth.append(calc_mean(azimuth)-90)
        results_elevation.append(calc_mean(elevation)-90)

    combined_azim.append(results_azimuth)
    combined_elev.append(results_elevation)

# Plot for azimuth range
fig = plt.figure(figsize=(8, 8), dpi=100)
ax = fig.add_subplot()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.grid(True)
ax.set_xlim(-95,95)
ax.set_ylim(-95,95)
ax.xaxis.set_label_coords(0, -0.05)
ax.yaxis.set_label_coords(-0.08, 0)
ax.set_ylabel('Measured azimuth [deg]', fontsize='18')
ax.set_xlabel('Set azimuth [deg]', fontsize='18')
ax.tick_params(labelsize=18)
ticks = np.arange(-90,92,30)
ax.xaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
ax.yaxis.set_ticks(ticks, [f"${i}\degree$" for i in ticks])
# ax.set_title("Antenna matrix azimuth range measurements", fontsize='20')
plot_ticks = [-90, -60, -45, -30, 0, 30, 45, 60]

mad_azim = []
median_azim = []
max_azim = []
min_azim = []
# max(combined_azim[i])

print(combined_azim[0])

for i in range(0, len(min_val)):
    mad_azim.append(median(abs(combined_azim[i]-median(combined_azim[i]))))


    max_azim.append(abs(max(combined_azim[i]) - plot_ticks[i]))
    min_azim.append(abs(min(combined_azim[i]) - plot_ticks[i]))
    median_azim.append(median(combined_azim[i]))
    
plt.plot([-90,90], [-90,90], 'b')
    
# ax.locator_params(nbins=4)

# ax = axs[0,1]
# ax.errorbar(plot_ticks, mad_azim, xerr=2, fmt='o')
# ax.set_title('Hor. symmetric')
plt.errorbar(plot_ticks, median_azim, yerr=[min_azim, max_azim], fmt='go')
# plt.plot(plot_ticks, median_azim, 'ro')


# for i in range(0, len(min_val)):
#     mad_azim = median(abs(combined_azim[i]-median(combined_azim[i])))
#     mad_elev = median(abs(combined_elev[i]-median(combined_elev[i])))
#     sd_azim = stdev(combined_azim[i])
#     sd_elev = stdev(combined_elev[i])
#     print(f"{i} Max Azimuth: {max(combined_azim[i]):.2f}")
#     print(f"{i} Min Azimuth: {min(combined_azim[i]):.2f}")
#     print(f"{i} Median Azimuth: {median(combined_azim[i]):.2f}")
#     print(f"{i} MAD Azimuth: {mad_azim:.2f}")

plt.subplots_adjust(left=0.11,
                    bottom=0.08,
                    right=0.95,
                    top=0.97,
                    wspace=0.15,
                    hspace=0.9)
# plt.tight_layout()
# plt.show()

with open('worst_case.npy', 'wb') as f:
    np.save(f, combined_azim[0])

with open('worst_case.npy', 'rb') as f:
    a = np.load(f)

    print(a)

plt.savefig("Median_vs_expected.jpg")

In [ ]:
# Power consumption plots CSV Default

import matplotlib.pyplot as plt
import numpy as np

data = np.genfromtxt('../current/default_1M_PHY.csv', delimiter=',')

avg_window = sum(data[:, 1])/len(data[:, 1])
avg_idle = sum(data[3000:7000, 1]/len(data[3000:7000, 1]))

fig = plt.figure(figsize=(20, 10), dpi=100)
ax = fig.add_subplot()
ax.grid(True)
ax.plot(data[:, 0]-data[0][0], data[:, 1]/1000)
ax.xaxis.set_label_coords(0, -0.08)
ax.yaxis.set_label_coords(-0.06, 0)

ax.text(
    70, 15, f"Avg. idle current: {avg_idle:.2f} uA", ha="center", va="center", rotation=0, size=30,
    bbox=dict(boxstyle="square,pad=0.4",fc="white", ec="k", lw=2))

ax.text(
    160, 15, f"Avg. current: {avg_window:.2f} uA", ha="center", va="center", rotation=0, size=30,
    bbox=dict(boxstyle="square,pad=0.4",fc="white", ec="k", lw=2))

ax.set_xlim(0,200)
ax.set_ylim(0,16)
ax.set_ylabel('Current draw [mA]', fontsize='30')
ax.set_xlabel('Time [ms]', fontsize='30')
ax.tick_params(labelsize=30)
ax.annotate('Idle period', (36, 8), fontsize='30')
rectangle = plt.Rectangle((25.5, 0), 48, 16,
facecolor="#90EE90", linewidth=7)
ax.add_patch(rectangle)
plt.savefig("1M_PHY_default.pdf")

In [ ]:
# Power consumption plots CSV Disabled modules

import matplotlib.pyplot as plt
import numpy as np

data = np.genfromtxt('../current/modules_disabled_1M_PHY.csv', delimiter=',')

avg_window = sum(data[:, 1])/len(data[:, 1])
avg_idle = sum(data[3000:7000, 1]/len(data[3000:7000, 1]))

fig = plt.figure(figsize=(20, 10), dpi=100)
ax = fig.add_subplot()
ax.grid(True)
ax.plot(data[:, 0]-data[0][0], data[:, 1]/1000)
ax.xaxis.set_label_coords(0, -0.08)
ax.yaxis.set_label_coords(-0.06, 0)

ax.text(
    70, 15, f"Avg. idle current: {avg_idle:.2f} uA", ha="center", va="center", rotation=0, size=30,
    bbox=dict(boxstyle="square,pad=0.4",fc="white", ec="k", lw=2))

ax.text(
    160, 15, f"Avg. current: {avg_window:.2f} uA", ha="center", va="center", rotation=0, size=30,
    bbox=dict(boxstyle="square,pad=0.4",fc="white", ec="k", lw=2))

ax.set_xlim(0,200)
ax.set_ylim(0,16)
ax.set_ylabel('Current draw [mA]', fontsize='30')
ax.set_xlabel('Time [ms]', fontsize='30')
ax.tick_params(labelsize=30)
ax.annotate('Idle period', (36, 8), fontsize='30')
rectangle = plt.Rectangle((26.5, 0), 48, 16,
facecolor="#90EE90", linewidth=7)
ax.add_patch(rectangle)
plt.savefig("1M_PHY_modules_disabled.pdf")
plt.show()